In [1]:
from keras import metrics
from keras.layers import BatchNormalization
from keras import metrics
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
import pickle
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras import layers
from keras.callbacks import EarlyStopping
import warnings, glob
import tensorflow as tf

2024-03-04 14:02:04.196508: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 14:02:04.196541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 14:02:04.197333: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 14:02:04.202246: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 14:02:04.894555: W tensorflow/compiler/tf2

In [2]:
warnings.filterwarnings('ignore')

In [3]:
def set_up_strategy():
    try:
        # Detect and initialize TPU
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        print('Running on TPU:', tpu.master())
    except ValueError:
        # Detect and initialize GPUs
        strategy = tf.distribute.MirroredStrategy()
        print('Running on multiple GPUs')
        tpu = None
    return strategy, tpu

In [4]:
# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy
strategy, tpu = set_up_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Running on multiple GPUs
REPLICAS: 1


2024-03-04 14:02:05.878301: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-04 14:02:05.878489: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-04 14:02:05.902521: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required l

In [5]:
import os

# Đường dẫn đến thư mục chứa dataset
data_dir = "../../SmartBin/data/raw_data/Garbage_classification_dataset"

# Định nghĩa các tham số cho dataset
batch_size = 8 * REPLICAS
img_size = (200, 200)
CHANNELS = 3
N_CLASSES = 5
TTA_STEPS = 0 # Do TTA if > 0


# Sử dụng image_dataset_from_directory để tạo dataset từ các hình ảnh trong thư mục đã cho
train_ds_original = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    batch_size=batch_size,
    image_size=img_size,
    validation_split=0.2,
    subset="training",
    seed=42,
    labels="inferred",
    label_mode="categorical")

val_ds_original = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    batch_size=batch_size,
    image_size=img_size,
    validation_split=0.2,
    subset="validation",
    seed=42,
    labels="inferred",
    label_mode="categorical")

Found 7188 files belonging to 6 classes.
Using 5751 files for training.
Found 7188 files belonging to 6 classes.
Using 1437 files for validation.


In [6]:
import random
import shutil
import pathlib
data_dir = pathlib.Path(data_dir)
class_names = train_ds_original.class_names
print(class_names)

['Bouteille_plastique', 'Brique_en_carton', 'Emballage_metallique', 'Ordure_mn++nagn++re', 'Papier_Carton', 'Verre']


In [7]:
for image_batch, labels_batch in train_ds_original:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(8, 200, 200, 3)
(8, 6)


In [8]:
image, label = next(iter(train_ds_original))

In [9]:
train_ds = train_ds_original.cache().shuffle(1000)
val_ds = val_ds_original.cache().shuffle(1000)

In [10]:
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds_original.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [11]:
def data_augment(image, label):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)

    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)

    return image, label

In [12]:
train_ds_augmented =  train_ds_original.map(data_augment)
val_ds_augmented =  val_ds_original.map(data_augment)

In [13]:
train_ds_combined = tf.data.Dataset.concatenate(train_ds_original, train_ds_augmented)
val_ds_combined = tf.data.Dataset.concatenate(val_ds_original, val_ds_augmented)

In [14]:
x_train = []
y_train = []
for images, labels in train_ds_combined:
    x_train.append(images)
    y_train.append(labels)

# After the loop, concatenate the lists into NumPy arrays
x_train = np.concatenate(x_train)
y_train = np.concatenate(y_train)


x_test= []
y_test = []
for images, labels in val_ds_combined:
    x_test.append(images)
    y_test.append(labels)

# After the loop, concatenate the lists into NumPy arrays
x_test = np.concatenate(x_test)
y_test = np.concatenate(y_test)

2024-03-04 14:02:07.679771: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: sRGB: out of place


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext] name: 

In [ ]:
y_test

In [ ]:
# for images,labels in val_ds_combined:
#     X_test = images

In [ ]:
# from sklearn.metrics import confusion_matrix
# import itertools

# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
# from keras.optimizers import RMSprop,Adam
# from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import ReduceLROnPlateau

# model = Sequential()
# #
# model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'Same', 
#                  activation ='relu', input_shape=(200, 200, 3)))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Dropout(0.25))
# #
# model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add(Dropout(0.25))
# # fully connected
# model.add(Flatten())
# model.add(Dense(256, activation = "relu"))
# model.add(Dropout(0.25))
# model.add(Dense(3, activation = "softmax"))



In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import keras

model = Sequential()
model.add(Conv2D(32, (5,5), activation='relu', input_shape=(200, 200, 3))),
model.add(BatchNormalization()),
model.add(MaxPooling2D(2, 2)),
model.add(Conv2D(64, (3,3), activation='relu')),
model.add(BatchNormalization()),
model.add(MaxPooling2D(2,2)),
model.add(Conv2D(128, (3,3), activation='relu')),
model.add(BatchNormalization()),
model.add(MaxPooling2D(2,2)),
model.add(Conv2D(256, (3,3), activation='relu')),
model.add(BatchNormalization()),
model.add(MaxPooling2D(2,2)),
model.add(Conv2D(512, (3,3), activation='relu')),
model.add(BatchNormalization()),
model.add(MaxPooling2D(2,2)),
model.add(Flatten()),
model.add(Dense(512, activation='relu')),
model.add(BatchNormalization()),
model.add(Dense(6, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# from keras.applications.vgg16 import VGG16
# base_model = VGG16(weights = "imagenet", include_top=False, 
#  input_shape = (200,200, 3))
# for layer in base_model.layers:
#  layer.trainable = False
# base_model.summary()

In [ ]:
# from keras.layers import Dense, Flatten
# from keras.models import Model
# last_layer = base_model.get_layer('block5_pool')
# last_output = last_layer.output
# x = Flatten()(last_output)
# x = Dense(3, activation='softmax', name='softmax')(x)

In [ ]:
# model = Model(inputs=base_model.input, outputs=x)
# model.summary() 

In [ ]:
# optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
# model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])


# datagen = ImageDataGenerator(
#         featurewise_center=False,  # set input mean to 0 over the dataset
#         samplewise_center=False,  # set each sample mean to 0
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#         samplewise_std_normalization=False,  # divide each input by its std
#         zca_whitening=False,  # dimesion reduction
#         rotation_range=5,  # randomly rotate images in the range 5 degrees
#         zoom_range = 0.1, # Randomly zoom image 10%
#         width_shift_range=0.1,  # randomly shift images horizontally 10%
#         height_shift_range=0.1,  # randomly shift images vertically 10%
#         horizontal_flip=False,  # randomly flip images
#         vertical_flip=False)  # randomly flip images

# datagen.fit()

# history = model.fit_generator(datagen.flow(train_ds_combined, batch_size=batch_size),
#                               epochs = 20, validation_data =val_ds_combined)


In [ ]:
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                            factor=0.2,
                                            mode = "min",
                                            min_lr=1e-12,
                                            patience=50,
                                            verbose=1)

early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                       mode= "min",
                                       patience=50,
                                       verbose=1,
                                       restore_best_weights=True)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
# optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
# model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(train_ds_combined, epochs=1,
                    validation_data=val_ds_combined,
                    callbacks=[early_stopping, reduce_lr],
                    verbose=1)

In [ ]:
model.evaluate(val_ds_combined)

In [ ]:
y_pred = model.predict(val_ds_combined)

In [ ]:
train_loss = history.history['loss']
train_acc = history.history['accuracy']
val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']

In [ ]:
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert y_test (one-hot encoded) to integer labels
y_true = np.argmax(y_test, axis=1)

# Generate classification report
print(classification_report(y_true, np.argmax(model.predict(x_test), axis=1)))

# Generate confusion matrix
print(confusion_matrix(y_true, np.argmax(model.predict(x_test), axis=1)))

In [ ]:
y_test.shape

In [ ]:
model.save('./model3classes.h5')

In [ ]:
print(x_train.dtype)